In [ ]:
import uuid

import astrocast.sim as sim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import importlib
from IPython.display import clear_output
import logging
import time
from pathlib import Path
import shutil
import humanize

In [ ]:
def generate_signal_function(N: int, burst_center: int, burst_width: float) -> callable:
    """
    Generates a signal function with a Gaussian burst.

    The function is N timepoints long, and each timepoint is an integer. The output is a float between 0 and 1.
    The signal starts and ends with 0 and has one smooth burst whose width and slope can be controlled by parameters.

    Args:
      N: The length of the signal in timepoints.
      burst_center: The center of the Gaussian burst (controls the position of the burst).
      burst_width: The width of the Gaussian burst (controls the spread of the burst).

    Returns:
      A function that takes a timepoint x and returns a value y, looping if x > N.
    """
    # Create a time series
    t = np.linspace(0, N - 1, N)
    
    # Normalize the Gaussian function so that its maximum is 1
    gaussian = np.exp(-((t - burst_center) ** 2) / (2 * burst_width ** 2))
    normalized_gaussian = gaussian / np.max(gaussian)
    
    # Define the signal function
    def signal_function(x: int) -> float:
        # Ensure the function loops
        x_mod = x % N
        return normalized_gaussian[x_mod]
    
    return signal_function

In [ ]:
importlib.reload(sim)
logging.basicConfig(level=100000000)

save_path = Path("exp")
if save_path.is_dir():
    shutil.rmtree(save_path)

dl = sim.DataLogger(save_path=save_path, save_checkpoint_interval=10, overwrite=True)

# , "repellent", "calcium"
env_dict = dict(molecules=["glutamate"], diffusion_rate=0.1,
                dt=25,  # 1
                degradation_factor=0.1)

N = 24  # Total timepoints
burst_center = int(N / 2)  # Center of the burst
burst_width = 0.5  # Width of the burst
signal_func = generate_signal_function(N, burst_center, burst_width)

grm_dict = dict(num_branches=32, z_thickness=3, jitter=2, release_amplitude=50,
                stochastic_probability=0,  # 0.1
                # signal_function=lambda x: 0 if x % 5 != 0 else 0.4,
                #signal_function=signal_func,
                )

ast_dict = dict(radius=3, num_branches=4, max_branch_radius=10,
                min_radius=0.001,
                start_spawn_radius=0.05, spawn_length=3, spawn_radius_factor=0.5,
                growth_factor=0.1, max_tries=5,
                glutamate_uptake_rate=0.1,
                glu_v_max=10, glu_k_m=500,
                allow_pruning=True,
                atp_cost_per_glutamate=-18, atp_cost_per_unit_surface=1,
                molecules=dict(glutamate=0, calcium=0, ATP=1e-6),
                trend_history=100,
                min_steepness=0.005, min_trend_amplitude=0.01,
                repellent_volume_factor=0, repellent_surface_factor=0,
                repellent_concentration=0  # 1
                )

my_sim = sim.Simulation(num_astrocytes=24, grid_size=(100, 100), border=5,
                        environment_param=env_dict, glutamate_release_param=grm_dict,
                        print_messages=True,
                        astrocyte_param=ast_dict, data_logger=dl)
my_sim.plot()

img_dir = Path("imgs/")
if img_dir.exists():
    shutil.rmtree(img_dir)
    img_dir.mkdir()
else:
    img_dir.mkdir()

my_sim.message_logger.save_messages(Path("log.txt"))

step_counter = 1

In [ ]:
from tqdm import tqdm

plotting = False
my_sim.message_logger.print_messages = False

for epoch in tqdm(range(int(1e7))):
    
    if plotting:
        clear_output(wait=True)
    
    # plotting
    my_sim.plot(last_n_messages=16 if plotting else None)
    my_sim.fig.savefig(img_dir.joinpath(f"img_{step_counter}.png"), dpi=(80))
    
    if plotting:
        display(my_sim.fig)
    
    # for step in range(max(2, int(np.log2(step_counter)))):
    t0 = time.time()
    for step in range(1):
        
        my_sim.run_simulation_step()
        step_counter += 1
        
        # export log
        my_sim.message_logger.save_messages(Path("log.txt"))

my_sim.plot(last_n_messages=16 if plotting else None)
my_sim.fig.savefig(img_dir.joinpath(f"img_{step_counter}.png"), dpi=(80))

In [ ]:
my_sim.message_logger.get_messages()

In [ ]:
importlib.reload(prep)

io = prep.IO()
arr = io.load(Path("imgs/"), lazy=False)
io.save(f"export_{uuid.uuid1().hex}.avi", data=arr, overwrite=True)

In [ ]:
# my_sim.plot(last_n_messages=15)
# display(my_sim.fig)
# my_sim.fig.savefig(img_dir.joinpath(f"img_{step_counter}.png"), dpi=(80))
# 
# my_sim.data_logger.save(Path("log.txt"))

In [ ]:
my_sim.astrocytes[0].plot_branch_history()


In [ ]:
branches = my_sim.astrocytes[2].branches

thickness = []
for branch in branches:
    avg_radius = np.mean(branch.start.radius + branch.end.radius)
    thickness.append(avg_radius)

print(f"{np.mean(thickness):.2f} +- {np.std(thickness):.2f}")

In [ ]:
thickness

In [ ]:
# fig, ax = plt.subplots(1, 1)
# ax.set_xscale('log')
# ax.set_yscale('log')
# _ = ax.hist(thickness, bins=1000)

In [ ]:
branch0 = None
branch_id = "92ede384"

branches = []
for ast in my_sim.astrocytes:
    branches += ast.branches

for branch in branches:
    if branch.get_short_id() == branch_id:
        branch0 = branch
        break

print(branch0)

if branch0 is not None:
    
    trend = branch0.get_trend("glutamate", intra=False)
    print(f"{trend:.1E}, {trend / ast_dict['min_trend_amplitude'] * 100:.1f}%")
    
    fig, ax = plt.subplots()
    
    hist = branch0.extracellular_history["glutamate"]
    ax.plot(hist)


In [ ]:
history = my_sim.environment_grid.history["glutamate"]

fig, ax = plt.subplots(1, 1)
ax.plot(history)

In [ ]:



# Example usage:
N = 24  # Total timepoints
burst_center = int(N / 2)  # Center of the burst
burst_width = 0.5  # Width of the burst
signal_func = generate_signal_function(N, burst_center, burst_width)

# Getting values from the signal function
x_len = N * 10
y_values = [signal_func(x) for x in range(x_len)]  # Example for 150 timepoints to show the looping

fig, ax = plt.subplots()
ax.plot(range(x_len), y_values)

In [ ]:
branches = []
for ast in my_sim.astrocytes:
    branches += ast.branches

glu_conc = []
glu_amount = []
for branch in branches:
    glu_conc.append(branch.get_concentration("glutamate"))
    glu_amount.append(branch.get_amount("glutamate"))

print(f"Conc: {np.mean(glu_conc):.2E} +- {np.std(glu_conc):.2E}")
print(f"Amount: {np.mean(glu_amount):.2E} +- {np.std(glu_amount):.2E}")

In [ ]:
glu_conc = my_sim.environment_grid.shared_arrays["glutamate"][0]
print(f"Conc: {np.mean(glu_conc):.2f} +- {np.std(glu_conc):.2f}")